In [ ]:
import functools
import os
from pathlib import Path
import sys

from dotenv import load_dotenv
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
from scipy.spatial import ConvexHull

# lowercase leap3d
sys.path.insert(0, '..')
from leap3d.config import DATA_DIR
from leap3d.scanning import ScanParameters
from leap3d.scanning import ScanResults
from leap3d.plotting import get_frames_for_temperature_cross_section_animation, plot_top_layer_temperature_at_timestep, get_frames_for_top_layer_temperatures

In [ ]:
ANIMATION_FRAME_DURATION = 1
GIF_FPS = 60
PLOT_DIR = Path("../Plots/")

In [ ]:
case_index = 0
case_filename = DATA_DIR / "case_0000.npz"
params_filename =  DATA_DIR / "Params.npy"
rough_coord_filename =  DATA_DIR / "Rough_coord.npz"

In [ ]:
X_MIN, X_MAX = -0.0012, 0.0012
Y_MIN, Y_MAX = -0.0012, 0.0012
Z_MIN, Z_MAX = -0.0006, 0
SAFETY_OFFSET = 0.0005
MELTING_POINT = 1723

In [ ]:
case_params = ScanParameters(params_filename, case_index,
                             x_min=X_MIN, x_max=X_MAX,
                             y_min=Y_MIN, y_max=Y_MAX,
                             z_min=Z_MIN, z_max=Z_MAX,
                             safety_offset=SAFETY_OFFSET,
                             melting_point=MELTING_POINT)
case_params

In [ ]:
case_results = ScanResults(case_filename, rough_coord_filename)

In [ ]:
fig, ims = get_frames_for_top_layer_temperatures(case_results, case_params)
fig.get_axes()[0].set_title("Top layer view of temperature map over time")
ani = animation.ArtistAnimation(fig, ims, interval=ANIMATION_FRAME_DURATION, blit=True,
                                repeat_delay=1000)
ani.save(PLOT_DIR / "top_heatmap.gif", fps=GIF_FPS)

In [ ]:
# fig, ims = get_frames_for_top_layer_temperatures(case_results, case_params, show_only_melt=True)
# fig.get_axes()[0].set_title("Top layer view of melt pool over time")
# ani = animation.ArtistAnimation(fig, ims, interval=ANIMATION_FRAME_DURATION, blit=True,
#                                 repeat_delay=1000)
# ani.save(PLOT_DIR / "top_melt.gif", fps=GIF_FPS)

In [ ]:
fig, ims = get_frames_for_temperature_cross_section_animation(case_results, case_params)

In [ ]:
fig.get_axes()[0].set_title("Cross section of temperature map over time")
ani = animation.ArtistAnimation(fig, ims, repeat=True, interval=ANIMATION_FRAME_DURATION, blit=True, repeat_delay=1000)
ani.save(PLOT_DIR / "temperature_cross_section.gif", fps=GIF_FPS, progress_callback=print)


In [ ]:
# fig, ims = get_frames_for_temperature_cross_section_animation(case_results, case_params, show_only_melt=True)

In [ ]:
# fig.get_axes()[0].set_title("Cross section of melt pool over time")
# ani = animation.ArtistAnimation(fig, ims, repeat=True, interval=ANIMATION_FRAME_DURATION, blit=True, repeat_delay=1000)
# ani.save(PLOT_DIR / "melt_cross_section.gif", fps=GIF_FPS, progress_callback=print)

In [ ]:
coordinates, temperature = case_results.get_melt_pool_coordinates_and_temperature(1800)

coordinates = np.array(coordinates)
hull = ConvexHull(coordinates)

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

for s in hull.simplices:
    s = np.append(s, s[0])
    ax.plot(coordinates[s, 0], coordinates[s, 1], coordinates[s, 2], "r-")

# Make axis label
for i in ["x", "y", "z"]:
    eval("ax.set_{:s}label('{:s}')".format(i, i))

# ax.set_xlim(case_params.x_min, case_params.x_max)
# ax.set_ylim(case_params.y_min, case_params.y_max)
# ax.set_zlim(case_params.z_min, case_params.z_max)

plt.show()